In [1]:
import streamlit as st
import pandas as pd
import numpy as np


In [2]:
st.text('bmi calculator')

2023-06-15 12:01:48.099 
  command:

    streamlit run c:\Users\anshul\miniconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [5]:
weight_in_kgs =1
height_in_mtr =1
age=1
#sex=male (1) or female(2)
sex=1
BMI = (weight_in_kgs)/(height_in_mtr)**2	



In [6]:
if BMI <16:
    print('under weight')
elif BMI <17 and BMI >=16:
    print('under weight')
elif BMI <18.5 and BMI >=17:
    print('under weight')
elif BMI <25 and BMI >=18.5:
    print('normal')
elif BMI <30 and BMI >=25:
    print('over weight')
else :
    print('obesity')


under weight


In [7]:
### BMR (kcal/day)

if sex == 1:
    BMR = 10 * weight_in_kgs + 6.25 *height_in_mtr*1000 - 5 *age  + 5
    print('kcal req per day {}'.format(BMR))
else :
    BMR = 10 * weight_in_kgs + 6.25 *height_in_mtr*1000 - 5 *age  -161
    print('kcal req per day {}'.format(BMR))

kcal req per day 6260.0
